In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

In [2]:
sc = MinMaxScaler()

In [3]:
train = sc.fit_transform(pd.read_csv('./Google_Stock_Price_Train.csv')['Open'].values.reshape(-1, 1))
test = sc.transform(pd.read_csv('./Google_Stock_Price_Test.csv')['Open'].values.reshape(-1, 1))

In [4]:
X_train = np.array([train[a: a + 60] for a in range(train.size - 60)]).reshape(1198, 60)
y_train = np.array([train[a + 60] for a in range(train.size - 60)])

In [5]:
total = pd.concat([pd.DataFrame(train), pd.DataFrame(test)]).values.reshape(1278)

In [6]:
X_test = np.array([total[a - 60: a] for a in range(1258, 1278)])
y_test = np.array([total[a] for a in range(1258, 1278)])

In [7]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)
results1 = pd.DataFrame()
results1['Price'] = sc.inverse_transform(y_test.reshape(-1, 1)).flatten()
results1['Predictions'] = sc.inverse_transform(model.predict(X_test)).flatten()

In [8]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [9]:
RNN = Sequential([
    LSTM(50, return_sequences = True), Dropout(0.2),
    LSTM(50, return_sequences = True), Dropout(0.2),
    LSTM(50, return_sequences = True), Dropout(0.2),
    LSTM(50, return_sequences = False), Dropout(0.2),
    Dense(1),
])

In [10]:
RNN.compile(Adam(), MeanSquaredError())

In [11]:
RNN.fit(X_train, y_train, 32, 150)

Epoch 1/150
38/38 [==============================] - 7s 42ms/step - loss: 0.0481
Epoch 2/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0067
Epoch 3/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0056
Epoch 4/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0059
Epoch 5/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0056
Epoch 6/150
38/38 [==============================] - 2s 43ms/step - loss: 0.0049
Epoch 7/150
38/38 [==============================] - 2s 43ms/step - loss: 0.0043
Epoch 8/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0047
Epoch 9/150
38/38 [==============================] - 2s 43ms/step - loss: 0.0046
Epoch 10/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0042
Epoch 11/150
38/38 [==============================] - 2s 42ms/step - loss: 0.0039
Epoch 12/150
38/38 [==============================] - 2s 43ms/step - loss: 0.0039
Epoch 13/150
38/38 [=====

In [12]:
results = pd.DataFrame()
results['Predictions'] = sc.inverse_transform(RNN.predict(X_test)).flatten()
results['Price'] =  sc.inverse_transform(y_test.reshape(-1, 1)).flatten()

1/1 [==============================] - 1s 1s/step


In [13]:
px.line(results)

In [14]:
px.line(results1)